In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Embedding, Input, Flatten
from tensorflow.keras.models import Model
import random
import pandas as pd
import numpy as np
import awkward as ak
import matplotlib as mpl
import matplotlib.pyplot as plt
import mplhep as hep
import boost_histogram as bh

from sklearn.metrics import classification_report, roc_auc_score
from scipy import interpolate
from sklearn.metrics import roc_curve, auc
from cycler import cycler
import awkward as ak

import uproot ## means uproot4
import sklearn.metrics as m
# from keras.layers import Input, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

2022-12-27 18:01:11.921195: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-27 18:01:11.921239: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/pku/zhaoyz/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [3]:

use_helvet = False ## true: use helvetica for plots, make sure the system have the font installed
if use_helvet:
    CMShelvet = hep.style.CMS
    CMShelvet['font.sans-serif'] = ['Helvetica', 'Arial']
    plt.style.use(CMShelvet)
else:
    plt.style.use(hep.style.CMS)

In [4]:

CustNanoData = {
    'Data' : "/data/pubfs/zhaoyz/Tree/V5/Merged/2018/Data/Tree_Data.root",            
    'QCD' : "/data/pubfs/zhaoyz/Tree/V5/Merged/2018/MC/Tree_QCD.root",   
    'BKG' : "/data/pubfs/zhaoyz/Tree/V5/Merged/2018/MC/Tree_BKG.root",           
    'Signal' : "/data/pubfs/zhaoyz/Tree/V5/Merged/2018/Signal_Nomatching/Tree_Total.root",

    'GluGlu' : "/data/pubfs/zhaoyz/Tree/V5/Merged/2018/Signal_Nomatching/Tree_GluGlu.root",            
    'VH' : "/data/pubfs/zhaoyz/Tree/V5/Merged/2018/Signal_Nomatching/Tree_VH.root",            
    'VBF' : "/data/pubfs/zhaoyz/Tree/V5/Merged/2018/Signal_Nomatching/Tree_VBF.root",            
    'ttH' : "/data/pubfs/zhaoyz/Tree/V5/Merged/2018/Signal_Nomatching/Tree_ttH.root",            
            
}
events = {typefile : {} for typefile in CustNanoData}
for typefile in CustNanoData:
    events[typefile] = uproot.lazy({CustNanoData[typefile]: "PKUTree" }) ## lazy means lazy computation style

In [5]:
#Function to combine the "signal" and "BKG" together.
def combine_events(eventsSignal, eventsBKG, split_step_signal = 1, split_step_bkg = 1):
    eventsSignal["isSignal"] = 1
    eventsBKG["isSignal"] = 0
    events_combine = {branch : ak.highlevel.Array for branch in eventsSignal.fields if ("hid" in branch or "isSignal" in branch)}
    #only the branch with "hid" and "isSignal" is needed.
    for branch in events_combine.keys():
        events_combine[branch] =  ak.concatenate([eventsSignal[branch][::split_step_signal],  eventsBKG[branch][::split_step_bkg]], highlevel=True)
        #Select the signal events and bkg events every split_step_signal'th events, and then connect the signal and background together.
    return events_combine
    


In [6]:
events_combine = combine_events(events["Signal"],events["BKG"], split_step_signal = 10, split_step_bkg = 10)

In [8]:
type(events_combine)

dict

In [9]:
type(events_combine["isSignal"])

awkward.highlevel.Array

In [10]:
type(events_combine["a_hidNeuron000"])

awkward.highlevel.Array

In [12]:
# We want to creat an array like [events["hid000"],... , events["hid127"]], so we can use it as the input of neural network.
DefineBranch = "events_combine[\"a_hidNeuron000\"]"
X_list = []
for i in range(128):
    if i < 10:
        vari = (DefineBranch.replace("000","00" + str(i)))
    elif i < 100:
        vari = (DefineBranch.replace("000","0" + str(i)))
    else:
        vari = (DefineBranch.replace("000",str(i)))
    X_list.append(vari)
print(X_list)


['events_combine["a_hidNeuron000"]', 'events_combine["a_hidNeuron001"]', 'events_combine["a_hidNeuron002"]', 'events_combine["a_hidNeuron003"]', 'events_combine["a_hidNeuron004"]', 'events_combine["a_hidNeuron005"]', 'events_combine["a_hidNeuron006"]', 'events_combine["a_hidNeuron007"]', 'events_combine["a_hidNeuron008"]', 'events_combine["a_hidNeuron009"]', 'events_combine["a_hidNeuron010"]', 'events_combine["a_hidNeuron011"]', 'events_combine["a_hidNeuron012"]', 'events_combine["a_hidNeuron013"]', 'events_combine["a_hidNeuron014"]', 'events_combine["a_hidNeuron015"]', 'events_combine["a_hidNeuron016"]', 'events_combine["a_hidNeuron017"]', 'events_combine["a_hidNeuron018"]', 'events_combine["a_hidNeuron019"]', 'events_combine["a_hidNeuron020"]', 'events_combine["a_hidNeuron021"]', 'events_combine["a_hidNeuron022"]', 'events_combine["a_hidNeuron023"]', 'events_combine["a_hidNeuron024"]', 'events_combine["a_hidNeuron025"]', 'events_combine["a_hidNeuron026"]', 'events_combine["a_hidNeuro

In [13]:
X = [eval(i) for i in X_list]

In [14]:
y = events_combine["isSignal"]
#We want to use y as the training output label, 1 = signal, 0 = bkg.

# Try to train a multi input NN

In [15]:
from tensorflow.keras.layers import Embedding, Input, Flatten
from tensorflow.keras.models import Model


a = ["input%s = keras.layers.Input(shape=(1,))"%(i) for i in range(128)]
for i in a:
    exec(i)
# define the input000-input127 one by one.

in_list = [eval("input%s"%(i)) for i in range(128)]
middle_concatenation = keras.layers.concatenate(in_list, axis=1)
# Connect input000-input127 together to form a list as input layer.

added1 = keras.layers.Dense(8, activation='relu')(middle_concatenation)
out = keras.layers.Dense(1, activation='sigmoid')(added1)

model = keras.models.Model(inputs=in_list, outputs=out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=30, batch_size=1000) #start to train.
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))
model.save("HWWmodel_Method2_MultiInput_4.h5")
print("Saved model to disk")

2022-12-27 18:09:53.464183: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-27 18:09:53.464213: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-27 18:09:53.464241: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (farm.phy.pku.edu.cn): /proc/driver/nvidia/version does not exist
2022-12-27 18:09:53.464490: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'awkward.highlevel.Array'>"}), <class 'awkward.highlevel.Array'>

In [ ]:
from tensorflow.keras.models import load_model
# load model
model_loaded = load_model('HWWmodel.h5')
# summarize model.
model_loaded.summary()
